In [1]:
#| default_exp DGRec_model

In [ ]:
!pip install tensorflow

# DGRec_model

> A module to generate and estimate likelihoods of TR to VR generation using a Long-Short Term Memory (LSTM) model.

In [11]:
#| hide
from nbdev.showdoc import show_doc
import os
#import tensorflow as tf
import numpy as np
#from tensorflow.keras.utils import to_categorical
#from tqdm import tqdm
#from tensorflow.keras import layers, models

In [27]:
#| hide
#from pathlib import Path

#data_path = Path("../../dgrec/example_data")

In [28]:
#| export

# One-hot encoding function for DNA sequences (A, C, G, T)
def one_hot_encode(sequence, vocab_size=4):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    integer_encoded = [mapping[base] for base in sequence]
    onehot_encoded = to_categorical(integer_encoded, num_classes=vocab_size)
    return onehot_encoded

def one_hot_decode(encoded_sequence):
    # Reverse the mapping used in encoding
    reverse_mapping = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}
    
    # Get the index of the max value in each one-hot vector (which corresponds to the base index)
    integer_decoded = np.argmax(encoded_sequence, axis=1)
    
    # Convert the integer sequence back to the DNA bases using the reverse mapping
    decoded_sequence = ''.join([reverse_mapping[i] for i in integer_decoded])
    
    return decoded_sequence

def separate_model(model): 
    # model: the lstm model of two layers used for predicting TR to VR
    # The present function is for disentangling the two layers to improve the generation speed of VR given TR
    weights = model.get_weights()
    
    # Define DNA alphabet size and model parameters
    vocab_size = 4  # e.g., A, T, C, G
    lstm_units = 8  # Number of LSTM units
    
    
    # Input layers (for the original model)
    inputs = layers.Input(shape=(None, vocab_size))
    inputs_mut = layers.Input(shape=(None, 2 * vocab_size + 1))  # Mutation input
    
    # Initialize initial states (h and c)
    initial_h = layers.Input(shape=(lstm_units,))  # Hidden state
    initial_c = layers.Input(shape=(lstm_units,))  # Cell state
    
    # First Bidirectional LSTM layer (Separate from second LSTM)
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(inputs)
    outputs = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))(x)
    
    # Extract the weights for the Bidirectional LSTM and TimeDistributed Dense Layer
    bidirectional_lstm_weights = model.layers[1].get_weights()  # Assuming first LSTM layer is at index 1
    time_distributed_dense_weights = model.layers[3].get_weights()  # Assuming TimeDistributed dense is at index 3
    
    outputs2 = layers.Input(shape=(None, vocab_size))
    # Second LSTM layer with return_state=True, passing initial states
    inputs2 = layers.Concatenate()([inputs, inputs_mut, outputs2])  # Concatenate inputs
    lstm_outputs, state_h, state_c = layers.LSTM(
        lstm_units, return_sequences=True, return_state=True
    )(inputs2, initial_state=[initial_h, initial_c])
    
    # Extract the weights for the second LSTM layer
    second_lstm_weights = model.layers[5].get_weights()  # Assuming second LSTM layer is at index 5
    
    # Final output layer
    final_outputs = layers.TimeDistributed(layers.Dense(vocab_size, activation='softmax'))(lstm_outputs)
    
    # Extract the weights for the final Dense layer
    final_dense_weights = model.layers[6].get_weights()  # Assuming final dense layer is at index 6
    
    # Rebuild the model with the separated layers
    # First, build the part for Bidirectional LSTM and TimeDistributed Dense Layer
    first_part_model = models.Model(inputs=inputs, outputs=outputs)

    first_part_model.set_weights([bidirectional_lstm_weights[0], bidirectional_lstm_weights[1], bidirectional_lstm_weights[2],  bidirectional_lstm_weights[3],  bidirectional_lstm_weights[4],  bidirectional_lstm_weights[5],
                                  time_distributed_dense_weights[0], time_distributed_dense_weights[1]])
    
    # Now, build the second part (second LSTM layer and final output layer)
    second_part_model = models.Model([inputs, inputs_mut,outputs2, initial_h, initial_c], [final_outputs, state_h, state_c])
    second_part_model.set_weights([second_lstm_weights[0], second_lstm_weights[1], second_lstm_weights[2],
                                   final_dense_weights[0], final_dense_weights[1]])
    return first_part_model,second_part_model
    
# Function to generate a sequence
def generate_sequence_from_onehot(X, firstmodel,secondmodel):
    """
    Generate a sequence iteratively using one cell of the second LSTM.
    
    Parameters:
    - X: np.array, Input sequences (one-hot encoded, reversed).
    - model: Trained Keras model for generation.
    - vocab_size: int, Size of the vocabulary (e.g., 4 for A, T, C, G).
    
    Returns:
    - generated_sequence: np.array, Generated sequence (one-hot encoded).
    """
    
    # Extract batch size and input sequence dimensions
    size = X.shape[0]  # Batch size
    seq_length = X.shape[1]  # Input sequence length
    vocab_size = X.shape[2]  # Input sequence length
    lstm_units=8
    # Initialize mutation input and output sequence arrays
    X_mut = np.zeros((size, 1, vocab_size * 2 + 1))  # Mutation input
    generated_sequence = np.zeros((size, seq_length, vocab_size))  # Output sequence array
    
    # Initialize the mutation input (start token for first step)
    X_mut[:, 0, -1] = 1  # Special start token in mutation input
    
    # Initial LSTM state
    initial_h = np.zeros((size, lstm_units))  # Hidden state
    initial_c = np.zeros((size, lstm_units))  # Cell state

    # Initialize the first LSTM block output (Bidirectional LSTM)
    first_layer_output = np.array(firstmodel.predict(X))
     # Use the last output from the first LSTM block to start generation
    # Iterative sequence generation
    for t in tqdm(range(seq_length), desc="Generating sequence"):
        # Concatenate the inputs for the second LSTM layer
        # Get the output, hidden state (h), and cell state (c) from the second LSTM
        output, new_h, new_c = secondmodel([X[:, t:t+1, :], X_mut, first_layer_output[:, t:t + 1, :],initial_h,initial_c])
        # Sample the next nucleotide (one-hot vector) based on the predicted probabilities
        next_word_idx = np.array([np.random.choice(vocab_size, p=dist) for dist in np.array(output)[:, 0, :]])
        next_word_one_hot = np.zeros((size, vocab_size))
        next_word_one_hot[np.arange(size), next_word_idx] = 1
        next_word_one_hot = np.expand_dims(next_word_one_hot, axis=1)  # Shape: (batch_size, 1, vocab_size)

        # Store the generated nucleotide in the sequence
        generated_sequence[:, t:t+1, :] = next_word_one_hot

        # Update the initial states for the next timestep
        initial_h, initial_c = np.array(new_h), np.array(new_c)

        # Update the mutation input for the next timestep
        if t < seq_length - 1:
            X_mut[:, 0, vocab_size:2*vocab_size] = next_word_one_hot[:, 0, :]  # Use generated nucleotide as input mutation
            X_mut[:, 0, :vocab_size] = X[:, t, :]  # Use original sequence as mutation input
            X_mut[:, 0, -1] = 0
    return generated_sequence

def sequences_same_length(sequences):
    """
    Check if all sequences in a list have the same length.
    
    Parameters:
    - sequences: list of sequences (e.g., strings, lists, or arrays)
    
    Returns:
    - bool: True if all sequences have the same length, False otherwise.
    """
    # Get the length of the first sequence
    first_length = len(sequences[0])

    # Check if all sequences have the same length
    for seq in sequences:
        if len(seq) != first_length:
            return False

    return True

#model_TR_to_VR = tf.keras.models.load_model(data_path/'LSTM_model.keras') #the LSTM model
#firstmodel,secondmodel=separate_model(model_TR_to_VR)

In [31]:
#| export

def generate_sequences(X_seq): #X _seq is a list of sequences ATCG sequences (faster if same length)
    """
    Generate list of VR from list of TR (one TR-> one VR).
    Parameters:
    - X_seq: list of TR sequences (e.g., strings, lists, or arrays)
    Returns:
    - list: list of VR sequence strings given TR sequences (corresponding to the initial TR list).
    """
    X=np.array([one_hot_encode(x[::-1]) for x in X_seq])
    if sequences_same_length(X_seq): # check that all sequences have same length: allow for fast generation
        Y=generate_sequence_from_onehot(X,firstmodel,secondmodel)
    else:
        Y=[generate_sequence_from_onehot(np.array([x]),firstmodel,secondmodel)[0] for x in X]
    return [one_hot_decode(y)[::-1] for y in Y] # return sequence in good order

def generate_sequences_oneTR(TR,n=1000):
    """
    Generate list of VR from one TR (one TR-> n VR).
    Parameters:
    - TR: one TR sequence (e.g., strings, lists, or arrays)
    -n: integer corresponding to the number of VR to generate
    Returns:
    - list: list of n VR sequence strings given the one TR sequence.
    """
    TR_list=[TR]*n
    return generate_sequences(TR_list)
    
def to_tensor_inputs(*args):
    return [tf.convert_to_tensor(x) for x in args]
    
def compute_likelihood(TR, VR):
    """
    Compute log-likelihood of VR from TR.

    TR, VR: same-length strings
    Returns: list of log-likelihoods
    """
    assert len(TR) == len(VR), "Mismatched lengths"

    vocab_size = 4
    lstm_units = 8
    seq_length = len(TR)

    # Encode and reverse sequences
    X = np.array([one_hot_encode(TR[::-1])])
    Y = np.array([one_hot_encode(VR[::-1])])

    # First model output
    first_output = firstmodel.predict(X, verbose=0)

    # Build mutation input (start with all start tokens)
    X_mut = np.zeros((1, seq_length, 2 * vocab_size + 1))
    X_mut[:, 0, -1] = 1  # Start tokens

    # Fill mutation input from known inputs
    for t in range(1, seq_length):
        X_mut[:, t, :vocab_size] = X[:, t - 1, :]
        X_mut[:, t, vocab_size:2*vocab_size] = Y[:, t - 1, :]

    # Initial LSTM states
    initial_h = np.zeros((batch_size, lstm_units))
    initial_c = np.zeros((batch_size, lstm_units))

    # Run full sequence in one call
    outputs, _, _ = secondmodel.predict([X, X_mut, first_output, initial_h, initial_c], verbose=0)

    # Compute log-likelihoods
    log_likelihoods = []
    for b in range(1):
        log_prob = 0.
        for t in range(seq_length):
            prob = outputs[b, t, np.argmax(Y[b, t])]
            log_prob += np.log(prob + 1e-8)
        log_likelihoods.append(log_prob)

    return log_likelihoods[0]

def compute_likelihood_batch(TR_batch, VR_batch):
    """
    Compute log-likelihoods of generating each VR[i] from TR[i] (batched).

    TR_batch, VR_batch: list of same-length strings
    Returns: list of log-likelihoods
    """
    assert all(len(tr) == len(vr) for tr, vr in zip(TR_batch, VR_batch)), "Mismatched lengths"

    vocab_size = 4
    lstm_units = 8
    batch_size = len(TR_batch)
    seq_length = len(TR_batch[0])

    # Encode and reverse sequences
    X = np.array([one_hot_encode(tr[::-1]) for tr in TR_batch])
    Y = np.array([one_hot_encode(vr[::-1]) for vr in VR_batch])

    # First model output
    first_output = firstmodel.predict(X, verbose=0)

    # Build mutation input (start with all start tokens)
    X_mut = np.zeros((batch_size, seq_length, 2 * vocab_size + 1))
    X_mut[:, 0, -1] = 1  # Start tokens

    # Fill mutation input from known inputs
    for t in range(1, seq_length):
        X_mut[:, t, :vocab_size] = X[:, t - 1, :]
        X_mut[:, t, vocab_size:2*vocab_size] = Y[:, t - 1, :]

    # Initial LSTM states
    initial_h = np.zeros((batch_size, lstm_units))
    initial_c = np.zeros((batch_size, lstm_units))

    # Run full sequence in one call
    outputs, _, _ = secondmodel.predict([X, X_mut, first_output, initial_h, initial_c], verbose=0)

    # Compute log-likelihoods
    log_likelihoods = []
    for b in range(batch_size):
        log_prob = 0.
        for t in range(seq_length):
            prob = outputs[b, t, np.argmax(Y[b, t])]
            log_prob += np.log(prob + 1e-8)
        log_likelihoods.append(log_prob)

    return log_likelihoods

def compute_likelihood_matrix(TR_list, VR_list, batch_size=64):
    """
    Compute log-likelihood matrix of generating each VR from each TR using batching.
    """
    result_matrix = []

    for i in range(0, len(TR_list)):
        row = []
        TR = TR_list[i]
        valid_VRs = [vr for vr in VR_list if len(vr) == len(TR)]

        if not valid_VRs:
            row = [-np.inf] * len(VR_list)
        else:
            batch_TRs = [TR] * len(valid_VRs)
            log_likelihoods = compute_likelihood_batch(batch_TRs, valid_VRs, firstmodel, secondmodel)
            idx = 0
            for vr in VR_list:
                if len(vr) != len(TR):
                    row.append(-np.inf)
                else:
                    row.append(log_likelihoods[idx])
                    idx += 1

        result_matrix.append(row)
    return result_matrix